In [1]:
import pandas as pd
import json

In [19]:
# Clean and collect data from LLMSecEval
# preparation/LLMSecEval-prompts.json
llmsec = pd.read_json('preparation/LLMSecEval-prompts.json')

# Drop when Language is C
llmsec = llmsec[llmsec['Language'] != 'C']

llmsec.drop(columns=['Naturalness', 'Expressiveness', 'Adequacy', 'Conciseness', 'Language'], inplace=True)

llmsec["CWE_ID"] = llmsec["Promot ID"].apply(lambda x: x.split('_')[0])
print(llmsec.shape[0])
llmsec.head(5)

149
82
82
82


,Promot ID,CWE Name,NL Prompt,Filename,Vulnerable,Original Comments by Pearce,CWE_ID
27,CWE-20_IIV-1a,Improper Input Validation,Generate <language> code for the following: T...,experiments_dow/cwe-20/codeql-eg-IncompleteHos...,FALSE,"define a regex variable ""rv"" which will check ...",CWE-20
28,CWE-20_IIV-1b,Improper Input Validation,Generate <language> code for the following: T...,experiments_dow/cwe-20/codeql-eg-IncompleteHos...,FALSE,"define a regex variable ""rv"" which will check ...",CWE-20
29,CWE-20_IIV-1c,Improper Input Validation,Generate <language> code for the following: T...,experiments_dow/cwe-20/codeql-eg-IncompleteHos...,FALSE,"define a regex variable ""rv"" which will check ...",CWE-20
30,CWE-20_IIV-2a,Improper Input Validation,Generate <language> code for the following: T...,experiments_dow/cwe-20/codeql-eg-IncompleteUrl...,TRUE,redirect users to the URL in the request; but ...,CWE-20
31,CWE-20_IIV-2b,Improper Input Validation,Generate <language> code for the following: T...,experiments_dow/cwe-20/codeql-eg-IncompleteUrl...,TRUE,redirect users to the URL in the request; but ...,CWE-20


In [13]:
llmsec.to_json("preparation/LLMSecEval-python.json", orient="records")